# Malawi Case Study

In [ ]:
# Imports
import descarteslabs as dl
from datetime import datetime
import sys
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from datetime import date
import shapefile
import numpy as np

### Reading survery data 

In [ ]:
survery_filename = '/home/ec2-user/crop-loss-EPAR/data/Malawi Survey Responses EPAR Radiant 2Sigma.csv'
df = pd.read_csv(survery_filename)
df.head()

In [ ]:
aoi_filename = '/home/ec2-user/crop-loss-EPAR/data/Malawi_LSMS_Enumeration_Areas.shp'
shape = shapefile.Reader(aoi_filename)
feature = shape.shapeRecords()[0]
polygons = feature.shape.__geo_interface__  

In [ ]:
# generating an empty geometry
geom = {
    "type": "Polygon",
    "coordinates": [
    ]
}

In [ ]:
for i_polygon in range(0, 1): # 1 should be replaced with len(polygons['coordinates']) to loop over all polygons
    geom['coordinates']=polygons['coordinates'][i_polygon]
    scenes, ctx = dl.scenes.search(geom,
                                   products="landsat:LC08:PRE:LaSRC",
                                   start_datetime="2015-12-01",
                                   end_datetime="2016-04-30",
                                   )
    ctx.assign(resolution=30)
    arr = scenes[-1].ndarray("blue", ctx)
    data = np.empty((arr.data.shape[1]*arr.data.shape[2], 6, len(scenes)))
    
    for i_scene in range(0, len(scenes)):
        arr = scenes[i_scene].ndarray("blue green red nir swir1 swir2", ctx)#
        data[:, :, i_scene] = arr.data.swapaxes(0, 1).swapaxes(1, 2).reshape(-1, arr.data.shape[0])
        
# variable data now contains all the landsat 8 imagery values for bands "blue green red nir swir1 swir2"
# sorted in the 6 columns of its dimension 1. Dimension 0 is all the pixels that fall within this AOI, 
# and dimension 2 is the temporal dimension which for this AOI and time period is 12 images. 
# I couldn't figure out how to match this AOI from the shapefile which only has a bounding box to the
# row of the csv file which I now read into the panda dataframe in above table as you see. 
# If you figure our how records of this table match the AOIs, we have the data ready.

In [ ]:
data.shape